In [1]:
!pip install --quiet langchain langchain_community transformers praw tiktoken faiss-cpu
!pip install --quiet sentence-transformers
!pip install -q transformers accelerate bitsandbytes sentencepiece optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from langchain.chains import LLMChain
import os
import json
from datetime import datetime
from getpass import getpass
import praw


In [3]:
client_id = getpass("Enter your Reddit client_id: ")
client_secret = getpass("Enter your Reddit client_secret: ")
user_agent = "persona-extractor-script by u/SwimmingIndividual61"

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)


Enter your Reddit client_id: ··········
Enter your Reddit client_secret: ··········


In [4]:
user = reddit.redditor("kojied")

data = []

for comment in user.comments.new(limit=100):
    data.append(comment.body)

for post in user.submissions.new(limit=100):
    text = post.title + "\n" + post.selftext
    data.append(text)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [5]:
combined_text = "\n\n".join(data)

In [6]:
combined_text

'Sorry to hear that man, in the future you may want to have multiple granaries, one close to your berries/deer/fish and one for your veggies/apples/honey.\nAlso I just store ale at the tavern or keep it in the brewery for this reason\n\nYeah but it doesn’t seem to affect much though\n\nIncredible stuff! How did you maintain character consistency? Did you generate a consistent character image and then use the image-to-video feature?\n\nI do too but I rarely finish a game haha. Too little late game content\n\nI do too but I rarely finish a game haha. Too little late game content\n\nYou have to kill the bandit units, or donate crop yields though the manor (tithing, different from taxation).\n\nThat proximity is ridiculous, deep mining, two fishing huts, vegetable, orchards, and trade iron for malt.\nimo trading raw materials is easier than trading crafted goods in the current build. High quantity stuff is easy to stock, and you could have multiple trading posts up without any major issues

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)


In [8]:
chunks = text_splitter.create_documents([combined_text])

In [9]:
chunks

[Document(metadata={}, page_content="Sorry to hear that man, in the future you may want to have multiple granaries, one close to your berries/deer/fish and one for your veggies/apples/honey.\nAlso I just store ale at the tavern or keep it in the brewery for this reason\n\nYeah but it doesn’t seem to affect much though\n\nIncredible stuff! How did you maintain character consistency? Did you generate a consistent character image and then use the image-to-video feature?\n\nI do too but I rarely finish a game haha. Too little late game content\n\nI do too but I rarely finish a game haha. Too little late game content\n\nYou have to kill the bandit units, or donate crop yields though the manor (tithing, different from taxation).\n\nThat proximity is ridiculous, deep mining, two fishing huts, vegetable, orchards, and trade iron for malt.\nimo trading raw materials is easier than trading crafted goods in the current build. High quantity stuff is easy to stock, and you could have multiple tradi

In [9]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/tmp/ipython-input-9-2815321582.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
vectordb=FAISS.from_documents(chunks,embedding=embeddings)

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `MY_TOKEN_HERE` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [12]:
model_id = "HuggingFaceTB/SmolLM3-3B"  #or you can use gpt-3.5-turbo model for better answers than this
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
    )

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

In [13]:
pipe=pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=300, do_sample=True, temperature=0.3)
llm=HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
/tmp/ipython-input-13-29712008.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe)


In [67]:
prompt = PromptTemplate(
    input_variables=["text"],
    template="""
You are a UX researcher creating a concise user persona based on Reddit posts by a single user.

Only analyze one user donot try to create persona of multiple users.. Do not repeat Reddit content.

Format:

Username: [Guess if not given]

Personality:
- [Each trait on its own line.]

Hobbies & Interests:
- [List separately with line breaks.]

Life Stage:
- [Short sentence.]

Goals & Motivations:
- [One point per line.]

Frustrations:
- [One point per line.]

Writing Style:
- [One line per observation.]

Demographic Clues:
- [Each clue on its own line.]

Citation:
- ["Short quote from the Reddit text."]

Only return the persona, clearly formatted. If the user is unclear, infer a realistic alias.
"""
)


In [68]:
qa = LLMChain(llm=llm, prompt=prompt)


In [69]:
combined_text = "\n\n".join(chunk.page_content.strip() for chunk in chunks[:13])
response = qa.invoke({
    "text": combined_text
})
print(response["text"].strip())


You are a UX researcher creating a concise user persona based on Reddit posts by a single user.

Only analyze one user donot try to create persona of multiple users.. Do not repeat Reddit content.

Format:

Username: [Guess if not given]

Personality:
- [Each trait on its own line.]

Hobbies & Interests:
- [List separately with line breaks.]

Life Stage:
- [Short sentence.]

Goals & Motivations:
- [One point per line.]

Frustrations:
- [One point per line.]

Writing Style:
- [One line per observation.]

Demographic Clues:
- [Each clue on its own line.]

Citation:
- ["Short quote from the Reddit text."]

Only return the persona, clearly formatted. If the user is unclear, infer a realistic alias.
username: TheRealMaverick

Personality:
- Enjoys intellectual discussions
- Has a sense of humor
- Prefers structured and logical thinking
- Values efficiency and precision
- Prefers clear and concise communication

Hobbies & Interests:
- Plays chess
- Engages in logical puzzles
- Reads science 

In [70]:
with open("persona_output.txt", "w", encoding="utf-8") as f:
    f.write(response["text"])

print("persona_output.txt")


persona_output.txt
